# Load models and predict

In [5]:
from lilac.jobs.job_base import BasicSeedJob

/Users/hikaru/github/ds-monorepo/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:

params={
    'train_path': 'data/train_v5.csv',
    'test_path': 'data/test_v5.csv',
    'features_dir': 'data/features/v5',
    'target_col': 'price_log',
    'base_col': 'area_log',
    'depth': 8,
    'min_child_samples': 200,
    'folds_gen_str': 'group',
    'group_key_col': 'ordered',
    'evaluator_str': 'mae',
    'unused_cols': ['id', 'ordered', 'built_year', 'area', 'nearest_min', 'age'],
    'target_enc_cols': ['nearest_sta', 'district'],
    'log_target_on_target_enc': False,
    'features_settings': [
        {
            'model_str': 'group',
            'params': {
                'group_keys': ['city', 'district', 'nearest_sta'],
                'agg_func_list': ['mean'],
                'input_cols': ['built_year_seireki','area','nearest_min','age','area_log','nearest_min_log','age_log']
                }
        },
        {
            'model_str': 'cat_lda',
            'params': {'main_col': 'city', 'sub_col': 'layout', 'num_topics': 5}
        },
        {
            'model_str': 'diff_group',
            'params': {'group_key': 'district',
            'input_cols': ['built_year_seireki', 'area_log', 'nearest_min_log']}
        }
    ],
    "pred_only": True
}

In [ ]:
data_path="data/models"
experiment_flag = "submit2"
job_names=["lgbm_diff_mae_42","lgbm_diff_mae_43","lgbm_mae_42","lgbm_mae_43"]
model_dirs = [f"{data_path}/{experiment_flag}/{job_name}" for job_name in job_names]
seeds=[42,43,42,43]
models = ["lgbm_diff_mae","lgbm_diff_mae","lgbm_mae","lgbm_mae"]

preds=[]
for model_dir, seed, model_str in zip(model_dirs,seeds,models):
    params["model_dir"]=model_dir
    params["seed"]=seed
    params["model_str"]= model_str
    output=BasicSeedJob(**params).run()
    pred=output["pred"]
    preds.append(pred)

In [ ]:
import numpy as np

pred=np.array(preds).mean(axis=0)

In [3]:
# 入賞した提出ファイルを読み込む
import pandas as pd

subs=pd.read_csv("data/subs/063.csv")

In [4]:
# subsとpredを比較


,ID,取引価格（総額）_log
0,1000117,6.767225
1,1000119,7.353071
2,1000379,7.726071
3,1000382,6.941047
4,1000385,6.548578
...,...,...
21002,47006115,6.960310
21003,47006186,7.400698
21004,47006549,7.394586
21005,47006712,7.514760
